In [1]:
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from NN_utils import *
import sys
sys.path.insert(0, "../")
from testsets import *
import evaluation

Using TensorFlow backend.


In [7]:
def evaluate(model, word_to_vec_mapping, test_data_file):
	ID_test, X_test, Y_test = csv_to_np(test_data_file[0])
	X_test_vec = average_word_embeddings(X_test, word_to_vec_mapping)
    
	predictions = model.predict(X_test_vec)
	pred_dict = dict()
	for i in range(len(predictions)):
		pred_dict[str(ID_test[i])] = label_to_sentiment(predictions[i])

	accuracy = model.score(X_test_vec, Y_test)
# 	loss, accuracy = model.evaluate(X_test_vec, Y_test_one_hot)

# 	print()
# 	print("Loss = ", loss)
	print("Test accuracy = " + str(accuracy*100) + "%")

	evaluation.evaluate(pred_dict, test_data_file[1], str(type(model).__name__))
	evaluation.confusion(pred_dict, test_data_file[1], str(type(model).__name__))

In [8]:
def test_model(model, word_to_vec_mapping):
	for testset in testsets:
		evaluate(model, word_to_vec_mapping, testset)
		# model.show_errors(testset[0])

In [11]:
'''Load data'''
train_data_file = "../data/processed/csv/proc-twitter-training-data.csv" 
dev_data_file = "../data/processed/csv/proc-twitter-dev-data.csv"
WORD_EMBEDDINGS_FILE = "../data/word_embeddings/glove.twitter.27B.50d.txt"

_, X_train, Y_train = csv_to_np(train_data_file)
_, X_dev, Y_dev = csv_to_np(dev_data_file)
word_to_vec_mapping = load_word_embeddings(WORD_EMBEDDINGS_FILE)


In [13]:
'''Averaged word embeddings features extraction'''
X_train = average_word_embeddings(X_train, word_to_vec_mapping)
X_dev = average_word_embeddings(X_dev, word_to_vec_mapping)

[-6.45643037e-02  5.35612479e-02  3.09630123e-01 -2.71337461e-02
  8.03587465e-02  1.49295871e-01  1.22052864e+00  3.73892486e-03
 -2.32599998e-01 -2.86851292e-02 -5.40953802e-02  5.74629754e-04
 -3.65668750e+00 -2.78849988e-02 -2.09466205e-02 -6.66052476e-02
  1.37302371e-01 -8.50084976e-02 -5.85137503e-01  3.88198117e-02
 -7.49707653e-02  2.27859379e-01  1.34269620e-01 -1.40196371e-01
 -1.47589247e-01  1.26439390e-01  1.36039946e-02 -2.07572501e-01
  3.23302863e-01 -2.77048124e-01  1.65206247e-01 -1.08705253e-01
 -8.30374542e-04  2.60607498e-01  7.56733751e-01 -1.21847522e-02
  1.81818750e-01 -4.63105753e-01  1.26562472e-02  1.61136251e-01
 -4.25754497e-01  5.36553748e-01 -8.17489795e-03  1.98133747e-01
  5.00932536e-02  1.38765501e-01  1.23891375e-01 -3.14427437e-02
 -4.09524992e-01 -1.11130626e-01]


In [14]:
'''GridSearchCV'''
grid_params = {'loss':['hinge', 'log', 'perceptron'], 'alpha': [1e-6, 5e-6, 0.00001, 0.00005, 0.0001, 0.0005]}
gridsearch = GridSearchCV(SGDClassifier(early_stopping=True, max_iter=1000, tol=1e-3), grid_params, cv=5, verbose=True, n_jobs=-1).fit(X_train, Y_train)
print("GridSearchCV training accuracy: ", gridsearch.score(X_train, Y_train))
print("GridSearchCV validation accuracy: ", gridsearch.score(X_dev, Y_dev))
print("Best parameters found: ", gridsearch.best_params_)
test_model(gridsearch, word_to_vec_mapping)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    6.7s finished


GridSearchCV training accuracy:  0.5953748253918982
GridSearchCV validation accuracy:  0.6065
Best parameters found:  {'alpha': 0.0005, 'loss': 'log'}
Test accuracy = 59.50155763239875%
../data/processed/txt/proc-twitter-test1.txt (GridSearchCV):
MacroF1: 0.470
            positive  negative  neutral
positive    0.631     0.076     0.293     
negative    0.142     0.650     0.208     
neutral     0.286     0.171     0.543     

Test accuracy = 64.22018348623854%
../data/processed/txt/proc-twitter-test2.txt (GridSearchCV):
MacroF1: 0.483
            positive  negative  neutral
positive    0.698     0.064     0.238     
negative    0.150     0.583     0.267     
neutral     0.354     0.129     0.518     

Test accuracy = 59.2686002522068%
../data/processed/txt/proc-twitter-test3.txt (GridSearchCV):
MacroF1: 0.463
            positive  negative  neutral
positive    0.647     0.083     0.270     
negative    0.196     0.515     0.288     
neutral     0.316     0.156     0.529     



In [15]:
'''LGR'''
LGR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, Y_train)
print("LGR Training accuracy: ", LGR.score(X_train, Y_train))
print("LGR Validation accuracy: ", LGR.score(X_dev, Y_dev))
test_model(LGR, word_to_vec_mapping)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LGR Training accuracy:  0.5992328329748786
LGR Validation accuracy:  0.6145
Test accuracy = 60.039648824695554%
../data/processed/txt/proc-twitter-test1.txt (LogisticRegression):
MacroF1: 0.458
            positive  negative  neutral
positive    0.688     0.046     0.266     
negative    0.124     0.671     0.205     
neutral     0.290     0.180     0.529     

Test accuracy = 63.680518078791145%
../data/processed/txt/proc-twitter-test2.txt (LogisticRegression):
MacroF1: 0.472
            positive  negative  neutral
positive    0.732     0.053     0.216     
negative    0.140     0.596     0.263     
neutral     0.370     0.131     0.499     

Test accuracy = 58.51197982345523%
../data/processed/txt/proc-twitter-test3.txt (LogisticRegression):
MacroF1: 0.432
            positive  negative  neutral
positive    0.694     0.074     0.232     
negative    0.191     0.525     0.284     
neutral     0.334     0.155     0.511     



In [16]:
'''Perceptron'''
Perceptron = Perceptron().fit(X_train, Y_train)
print("Perceptron Training accuracy: ", Perceptron.score(X_train, Y_train))
print("Perceptron Validation accuracy: ", Perceptron.score(X_dev, Y_dev))
test_model(Perceptron, word_to_vec_mapping)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Perceptron Training accuracy:  0.5067736857275892
Perceptron Validation accuracy:  0.514
Test accuracy = 47.578589634664404%
../data/processed/txt/proc-twitter-test1.txt (Perceptron):
MacroF1: 0.161
            positive  negative  neutral
positive    0.855     0.031     0.114     
negative    0.229     0.557     0.214     
neutral     0.394     0.157     0.449     

Test accuracy = 42.741500269832706%
../data/processed/txt/proc-twitter-test2.txt (Perceptron):
MacroF1: 0.165
            positive  negative  neutral
positive    0.904     0.007     0.089     
negative    0.345     0.448     0.207     
neutral     0.503     0.111     0.385     

Test accuracy = 46.44808743169399%
../data/processed/txt/proc-twitter-test3.txt (Perceptron):
MacroF1: 0.180
            positive  negative  neutral
positive    0.797     0.024     0.179     
negative    0.253     0.532     0.215     
neutral     0.420     0.146     0.434     



In [17]:
'''LinearSVC'''
LinearSVC = LinearSVC().fit(X_train, Y_train)
print("LinearSVC Training accuracy: ", LinearSVC.score(X_train, Y_train))
print("LinearSVC Validation accuracy: ", LinearSVC.score(X_dev, Y_dev))
test_model(LinearSVC, word_to_vec_mapping)

/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC Training accuracy:  0.5968825524932928
LinearSVC Validation accuracy:  0.6105
Test accuracy = 60.0113282356273%
../data/processed/txt/proc-twitter-test1.txt (LinearSVC):
MacroF1: 0.431
            positive  negative  neutral
positive    0.686     0.060     0.255     
negative    0.132     0.694     0.174     
neutral     0.287     0.181     0.532     

Test accuracy = 64.00431732325957%
../data/processed/txt/proc-twitter-test2.txt (LinearSVC):
MacroF1: 0.450
            positive  negative  neutral
positive    0.733     0.059     0.208     
negative    0.167     0.643     0.190     
neutral     0.367     0.130     0.503     

Test accuracy = 58.301807482135345%
../data/processed/txt/proc-twitter-test3.txt (LinearSVC):
MacroF1: 0.424
            positive  negative  neutral
positive    0.687     0.073     0.240     
negative    0.155     0.578     0.267     
neutral     0.335     0.158     0.507     

